In [2]:
import sys

if sys.platform == 'win32':
    filepath = r"D:\Code\唐宇迪课程资料\随机森林\data\temps_extended.csv"
else:
    filepath = r'/Users/wangrui/Desktop/Files/唐宇迪课程资料/第十六章：基于随机森林的气温预测/随机森林/data/temps_extended.csv'

# 数据读取
import pandas as pd

features = pd.read_csv(filepath)
features.head(5)

,year,month,day,weekday,ws_1,prcp_1,snwd_1,temp_2,temp_1,average,actual,friend
0,2011,1,1,Sat,4.92,0.00,0,36,37,45.6,40,40
1,2011,1,2,Sun,5.37,0.00,0,37,40,45.7,39,50
2,2011,1,3,Mon,6.26,0.00,0,40,39,45.8,42,42
3,2011,1,4,Tues,5.59,0.00,0,39,42,45.9,38,59
4,2011,1,5,Wed,3.80,0.03,0,42,38,46.0,45,39


In [3]:
# One Hot Encoding
features = pd.get_dummies(features)

# Extract features and labels
labels = features['actual']
features = features.drop('actual', axis = 1)

# List of features for later use
feature_list = list(features.columns)

# Convert to numpy arrays
import numpy as np

features = np.array(features)
labels = np.array(labels)

# Training and Testing Sets
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels,
                                                                            test_size = 0.25, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1643, 17)
Training Labels Shape: (1643,)
Testing Features Shape: (548, 17)
Testing Labels Shape: (548,)


# 调参

In [4]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 42)

from pprint import pprint

# 打印所有参数
pprint(rf.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


1、max_features = auto是什么意思？
- If “auto”, then max_features=n_features

In [5]:
from sklearn.model_selection import RandomizedSearchCV

# 建立树的个数
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# 最大特征的选择方式
max_features = ['auto', 'sqrt']
# 树的最大深度
max_depth = [int(x) for x in np.linspace(10, 20, num = 2)]
max_depth.append(None)
# 节点最小分裂所需样本个数
min_samples_split = [2, 5, 10]
# 叶子节点最小样本数，任何分裂不能让其子节点样本数少于此值
min_samples_leaf = [1, 2, 4]
# 样本采样方法
bootstrap = [True, False]

# Random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

疑问： RandomSearchCV是如何"随机搜索"的？

In [6]:
# 随机选择最合适的参数组合
rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                               n_iter=10, scoring='neg_mean_absolute_error',
                               cv=3, verbose=2, random_state=42, n_jobs=4)

# 执行寻找操作
rf_random.fit(train_features, train_labels)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=4,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [9]:
rf_random.best_estimator_

RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                      n_estimators=1600)

In [10]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape

    print('平均气温误差.',np.mean(errors))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

In [11]:
# 老模型
base_model = RandomForestRegressor( random_state = 42)
base_model.fit(train_features, train_labels)
evaluate(base_model, test_features, test_labels)

平均气温误差. 3.7161131386861315
Accuracy = 93.73%.


In [12]:
# 新模型
best_random = rf_random.best_estimator_
evaluate(best_random, test_features, test_labels)

平均气温误差. 3.6466876214501975
Accuracy = 93.84%.


In [13]:
best_random.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 10,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1600,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [14]:
feature_list

['year',
 'month',
 'day',
 'ws_1',
 'prcp_1',
 'snwd_1',
 'temp_2',
 'temp_1',
 'average',
 'friend',
 'weekday_Fri',
 'weekday_Mon',
 'weekday_Sat',
 'weekday_Sun',
 'weekday_Thurs',
 'weekday_Tues',
 'weekday_Wed']

### 利用RandomizedSearchCV查找大概范围，之后再用GridSearchCV来确定最终参数

来总结一下我们的调参任务吧：

- 1.参数空间是非常重要的，它会对结果产生决定性影响，所以在开始任务之前，得选择大致一个合适区间，可以参考一些相同任务论文的经验值。
- 2.随机搜索可以更节约时间，尤其是在任务开始阶段，我们并不知道哪一个参数在哪一个位置效果能更好，这样我们可以把参数间隔设置的更大一些，先用随机搜索确定一些大致位置。
- 3.网络搜索相当于地毯式搜索了，当我们得到了大致位置之后，想在这里寻找到最优参数的时候就派上用场了，可以把随机和网络搜索当做一套组合拳，搭配使用。
- 4.最后调参的方法其实还有很多的，比如贝叶斯优化，这个还是蛮有意思的，跟大家简单说一下，想一想我们之前的调参方式，是不是每一个都是独立的进行不会对之后的结果产生任何影响，贝叶斯优化的基本思想在于每一个优化都是在不断积累经验，这样我会慢慢得到最终的解应当在的位置，相当于前一步结果会对后面产生影响了，如果大家对贝叶斯优化感兴趣，可以参考下Hyperopt工具包，用起来也很简便：